<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/resnet_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn

In [10]:
#input -> 저장 -> 합성곱 3*3 -> 배치정규화 -> ReLU -> 합성곱 3*3 -> 배치정규화 -> skip conection -> ReLU

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_sizes = 3):
        super(BasicBlock, self).__init__()

        self.c1 = nn.Conv2d(in_channels,out_channels, kernel_size=kernel_sizes,padding=1)
        self.c2 = nn.Conv2d(out_channels,out_channels, kernel_size=kernel_sizes,padding=1)

        self.downsample = nn.Conv2d(in_channels,out_channels,kernel_size=1)

        self.bn1 = nn.BatchNorm2d(num_features = out_channels)
        self.bn2 = nn.BatchNorm2d(num_features = out_channels)

        self.relu = nn.ReLU()

    def forward(self, x):
        x_ = x

        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)

        #skip connection : 아웃풋으로 나온 채널수를 맞춤
        x_ = self.downsample(x_)
        x += x_
        x = self.relu(x)

        return x

In [11]:
class ResNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ResNet, self).__init__()

        self.b1 = BasicBlock(in_channels = 3, out_channels = 64)
        self.b2 = BasicBlock(in_channels = 64, out_channels = 128)
        self.b3 = BasicBlock(in_channels = 128, out_channels = 256)

        self.pool = nn.AvgPool2d(kernel_size = 2, stride = 2)

        #분류기
        self.fc1 = nn.Linear(in_features = 4096, out_features = 2048)
        self.fc2 = nn.Linear(in_features = 2048, out_features = 512)
        self.fc3 = nn.Linear(in_features = 512, out_features = num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.b1(x)
        x = self.pool(x)

        x = self.b2(x)
        x = self.pool(x)

        x = self.b3(x)
        x = self.pool(x)

        #평탄화
        x = torch.flatten(x, start_dim = 1)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)

        return x

In [12]:
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose
from torchvision.transforms import RandomCrop, RandomHorizontalFlip,Normalize
import torchvision.transforms as T

In [13]:
transforms = Compose([
    RandomCrop((32,32),padding=4),
    RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465),std=(0.247,0.243,0.261))
])

In [14]:
train_data = CIFAR10(root='./',train=True,download=True,transform=transforms)
test_data = CIFAR10(root='./',train=False,download=True,transform=transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ResNet(10)

model.to(device)

Files already downloaded and verified
Files already downloaded and verified


ResNet(
  (b1): BasicBlock(
    (c1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasicBlock(
    (c1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasicBlock(
    (c1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [15]:
# SummaryWriter 인스턴스 생성
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [16]:
lr = 1e-3

optim = Adam(model.parameters(),lr=lr)

for epoch in range(100):
    for data,label in train_loader:
        optim.zero_grad()

        preds = model(data.to(device))

        loss = nn.CrossEntropyLoss()(preds,label.to(device))
        loss.backward()
        optim.step()

    print(f'epoch : {epoch+1} loss:{loss.item()}')
    writer.add_scalar("Loss/train", loss.item(), epoch+1 )

epoch : 1 loss:2.3025853633880615
epoch : 2 loss:2.3025853633880615
epoch : 3 loss:2.3025853633880615
epoch : 4 loss:2.3025853633880615
epoch : 5 loss:2.3025853633880615
epoch : 6 loss:2.3025853633880615
epoch : 7 loss:2.3025853633880615
epoch : 8 loss:2.3025853633880615
epoch : 9 loss:2.3025853633880615
epoch : 10 loss:2.3025853633880615
epoch : 11 loss:2.3025853633880615
epoch : 12 loss:2.3025853633880615
epoch : 13 loss:2.3025853633880615
epoch : 14 loss:2.3025853633880615
epoch : 15 loss:2.3025853633880615
epoch : 16 loss:2.3025853633880615
epoch : 17 loss:2.3025853633880615
epoch : 18 loss:2.3025853633880615
epoch : 19 loss:2.3025853633880615
epoch : 20 loss:2.3025853633880615
epoch : 21 loss:2.3025853633880615
epoch : 22 loss:2.3025853633880615
epoch : 23 loss:2.3025853633880615
epoch : 24 loss:2.3025853633880615
epoch : 25 loss:2.3025853633880615
epoch : 26 loss:2.3025853633880615
epoch : 27 loss:2.3025853633880615
epoch : 28 loss:2.3025853633880615
epoch : 29 loss:2.30258536338

In [17]:
torch.save(model.state_dict(),'cifar-resnet.pth')

In [18]:
num_corr =  0

model.eval()
with torch.no_grad():
    for data,label in test_loader:
        pred = model(data.to(device))
        preds = pred.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr
num_corr/len(test_data)

0.1